In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
from astra.torch.models import EfficientNet,ResNetClassifier, MLP, MLPClassifier, EfficientNetClassifier
# from astra.torch.utils import count_params
# print(count_params(EfficientNetClassifier()))
# print(count_params(ResNetClassifier()))

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch.utils.data import random_split
from torchvision.utils import make_grid
import torch.nn as nn
import copy
import time
import os
import torchvision.models as models

import argparse
from torch.utils.data import Subset
import csv
device = torch.device('cuda')
from ResNet import *
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score,precision_score,recall_score
from sklearn.metrics import ConfusionMatrixDisplay

### transforms for downstream tasks

In [4]:
# transform_test = transforms.Compose([
#         # transforms.Resize(224),
#         # transforms.ToTensor(),
#         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
#     ])
# s=0.5
# color_jitter = transforms.ColorJitter(0.4*4, 0.4*s, 0.4*s, 0.2*s)
# transform_eval = transforms.Compose([
#         # transforms.RandomResizedCrop((224,224), scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333)),
#         transforms.RandomHorizontalFlip(p=0.5),
#         transforms.RandomApply([color_jitter], p=0.5), 
#         transforms.RandomGrayscale(p=0.4),
#         # transforms.Resize(224),
#         # transforms.ToTensor(),
#         # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
#     ])


In [5]:
loaded_data = torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/test_data.pt")
loaded_data1 = torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/data.pt")
images= loaded_data1['images']
labels = loaded_data1['labels']
images = images / 255
# images = (images - images.mean(dim=(0, 2, 3), keepdim=True)) / images.std(dim=(0, 2, 3), keepdim=True)
images1 = loaded_data['images']
labels1 = loaded_data['labels']
images1 = images1 / 255
# images1 = (images1 - images1.mean(dim=(0, 2, 3), keepdim=True)) / images1.std(dim=(0, 2, 3), keepdim=True)
# print(images1.shape)
# print(labels1.shape)
# print(images.shape)
# print(labels.shape)
train_images = images
train_labels = labels
test_images = images1
test_labels = labels1
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

torch.Size([25500, 3, 224, 224])
torch.Size([25500])
torch.Size([10025, 3, 224, 224])
torch.Size([10025])


In [6]:
# loaded_datad1= torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data_final/delhi_data_90.pt")
# imagesd= loaded_datad1['images']
# labelsd = loaded_datad1['labels']

# images1 = images1 / 255
# images1 = (images1 print(images1.shape)
# print(labels1.shape)- images1.mean(dim=(0, 2, 3), keepdim=True)) / images1.std(dim=(0, 2, 3), keepdim=True)


In [7]:
# train_images= torch.cat((train_images,imagesd),0)
# train_labels= torch.cat((train_labels,labelsd),0)
# print(train_images.shape)
# print(train_labels.shape)

In [8]:
train_images.dtype, train_labels.dtype, test_images.dtype, test_labels.dtype


(torch.float32, torch.uint8, torch.float32, torch.uint8)

In [9]:
# train_labels = train_labels.type(torch.uint8)
# test_labels = test_labels.type(torch.uint8)

In [10]:
# train_images.dtype, train_labels.dtype, test_images.dtype, test_labels.dtype


In [11]:
train_labels_count = (train_labels == 1).sum()
test_labels_count = (test_labels == 1).sum()

print("Number of train labels with value 1:", train_labels_count)
print("Number of test labels with value 1:", test_labels_count)
# test_images=transform_test(test_images)

Number of train labels with value 1: tensor(1697)
Number of test labels with value 1: tensor(1042)


### Data preporcessing for how much percentage of the data is used for finetune

In [12]:
# train_images=transform_eval(train_images)


In [13]:
# print(train_images.shape)
# print(test_images.shape)
train_set=torch.utils.data.TensorDataset(train_images,train_labels)
test_set=torch.utils.data.TensorDataset(test_images,test_labels)
train_loader=torch.utils.data.DataLoader(train_set,batch_size=512,shuffle=True,num_workers=8)
test_loader=torch.utils.data.DataLoader(test_set,batch_size=512,shuffle=False,num_workers=8)

In [14]:
len(train_set)

25500

In [15]:
base_encoder=torchvision.models.efficientnet_b0(pretrained=True)
# base_encoder
base_encoder.classifier[1]=nn.Linear(1280,2)
base_encoder.to(device)

/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

### fine-tune the downstram model  and predict on test data

In [16]:
class_0_weight=train_labels_count/len(train_images)
print(class_0_weight)
class_1_weight=1-class_0_weight
print(class_1_weight)

tensor(0.0665)
tensor(0.9335)


In [17]:
# nn.CrossEntropyLoss??

In [18]:

# criterion = nn.CrossEntropyLoss(weight=torch.tensor([class_0_weight, class_1_weight]).to(device))
criterion=nn.Focal
optimizer = torch.optim.Adam(base_encoder.parameters(), lr=3e-4)
train_losses = []
train_accuracies = []
train_precisions = []
train_recalls = []
train_f1_scores = []
test_f1_scores = []
test_losses = []
test_accuracies = []
test_precisions = []
test_recalls = []
for epoch in range(30):
    base_encoder.train()
    true_labels = []
    pred_labels = []        
    running_loss = 0.0
    for i ,(inputs, labels) in enumerate(train_loader):
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        optimizer.zero_grad()
        output = base_encoder(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(output, 1)
        true_labels.extend(labels.data.cpu().numpy())
        pred_labels.extend(preds.data.cpu().numpy())
    epoch_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_loss)
    train_accuracy = accuracy_score(true_labels, pred_labels)
    train_accuracies.append(train_accuracy)
    train_precision = precision_score(true_labels, pred_labels)
    train_precisions.append(train_precision)
    train_recall = recall_score(true_labels, pred_labels)
    train_recalls.append(train_recall)
    train_f1 = f1_score(true_labels, pred_labels)
    train_f1_scores.append(train_f1)
    print(f'Epoch {epoch}, Loss: {epoch_loss}, Accuracy: {train_accuracy}, Precision: {train_precision}, Recall: {train_recall}, F1: {train_f1}')

    with torch.no_grad():
        base_encoder.eval()
        true_labels = []
        pred_labels = []
        for i, (inputs, labels) in enumerate(test_loader):
            inputs = inputs.to('cuda')
            labels = labels.to('cuda')
            output = base_encoder(inputs)
            _, preds = torch.max(output, 1)
            true_labels.extend(labels.data.cpu().numpy())
            pred_labels.extend(preds.data.cpu().numpy())
    test_accuracy = accuracy_score(true_labels, pred_labels)
    test_accuracies.append(test_accuracy)
    test_precision = precision_score(true_labels, pred_labels)
    test_precisions.append(test_precision)
    test_recall = recall_score(true_labels, pred_labels)
    test_recalls.append(test_recall)
    test_f1 = f1_score(true_labels, pred_labels)
    test_f1_scores.append(test_f1)
    print(f'Test - Accuracy: {test_accuracy}, Precision: {test_precision}, Recall: {test_recall}, F1: {test_f1}')

        


Epoch 0, Loss: 0.21256689541830737, Accuracy: 0.8942745098039215, Precision: 0.3812693130496791, Recall: 0.9451974071891573, F1: 0.5433604336043361
Test - Accuracy: 0.8824937655860349, Precision: 0.4672131147540984, Recall: 0.9299424184261037, F1: 0.6219512195121951
Epoch 1, Loss: 0.0596610213728512, Accuracy: 0.9809019607843137, Precision: 0.7851083883129123, Recall: 0.9817324690630524, F1: 0.8724797067295104
Test - Accuracy: 0.9237905236907731, Precision: 0.5877525252525253, Recall: 0.8934740882917467, F1: 0.7090632140137091
Epoch 2, Loss: 0.03109229525280934, Accuracy: 0.9895686274509804, Precision: 0.8690046415678184, Recall: 0.9929286977018268, F1: 0.9268426842684268
Test - Accuracy: 0.9602992518703242, Precision: 0.78, Recall: 0.8608445297504799, F1: 0.8184306569343066
Epoch 3, Loss: 0.01767047964576997, Accuracy: 0.9937254901960785, Precision: 0.915180983252296, Recall: 0.9982321744254566, F1: 0.9549041713641487
Test - Accuracy: 0.9655860349127182, Precision: 0.850956696878147, 

In [35]:
result_dict = {}

In [36]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

percent = "5D%"
ssl = "NoSimCLR"
with torch.no_grad():    
    base_encoder.eval()
    correct = 0
    total = 0
    y_pred = []
    y_test = []
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            output = base_encoder(inputs)
            _, preds = torch.max(output, 1)
            total += targets.size(0)
            correct += preds.eq(targets).sum().item()
            y_pred.extend(preds.cpu().numpy())
            y_test.extend(targets.cpu().numpy())
    
    y_pred = np.array(y_pred)
    y_test = np.array(y_test)
    
    print('Test Acc: %.3f%% (%d/%d)' % (100.*correct/total, correct, total))
    print(len(y_pred), len(y_test))
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print("Accuracy: ", accuracy)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 Score: ", f1)

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:")
    print(cm)
    
    # If needed, class-specific metrics
    class_accuracy = cm.diagonal() / cm.sum(axis=1)
    class_precision = precision_score(y_test, y_pred, average=None)
    class_recall = recall_score(y_test, y_pred, average=None)
    class_f1 = f1_score(y_test, y_pred, average=None)

    print("Class-specific Accuracy:", class_accuracy)
    print("Class-specific Precision:", class_precision)
    print("Class-specific Recall:", class_recall)
    print("Class-specific F1 Score:", class_f1)
    result_dict[f"{percent} {ssl}"] = {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}
    print(result_dict)

Test Acc: 94.793% (9503/10025)
10025 10025
Accuracy:  0.9479301745635911
Precision:  0.8376623376623377
Recall:  0.6190019193857965
F1 Score:  0.7119205298013245
Confusion Matrix:
[[8858  125]
 [ 397  645]]
Class-specific Accuracy: [0.98608483 0.61900192]
Class-specific Precision: [0.95710427 0.83766234]
Class-specific Recall: [0.98608483 0.61900192]
Class-specific F1 Score: [0.97137844 0.71192053]
{'5D% NoSimCLR': {'accuracy': 0.9479301745635911, 'f1': 0.7119205298013245, 'precision': 0.8376623376623377, 'recall': 0.6190019193857965}}


: 

In [21]:
# y_pred,ytest=prdict(net_eval, test_loader, "30%", "ssl", device,)

In [22]:
import pandas as pd

In [23]:
# result_dict = {
#     "100% SimCLR": {
#         "Accuracy": 0.9012222222222223,
#         "Precision": 0.5627563576702215,
#         "Recall": 0.6583493282149712,
#         "F1 Score": 0.60111455108358
#     },
#     "100% SimCLR augmentaion": {
#         'Accuracy':  0.9066666666666666,
#         'Precision':  0.6185446009389671,
#         'Recall':  0.5057581573896354,
#         'F1 Score':  0.5564941921858501
#     },
#     "100% no ssl": {
#         'Accuracy':  0.9072222222222223,
#         'Precision':  0.6624803767660911,
#         'Recall':  0.4049904030710173,
#         'F1 Score':  0.5026801667659321
#     },
#     "100% SimCLR focal loss": {
#         'Accuracy':  0.9163333333333333,
#         'Precision':  0.7680890538033395,
#         'Recall':  0.39731285988483683,
#         'F1 Score':  0.523719165085389
#     }
# }


In [24]:
print(result_dict)

{'5D% NoSimCLR': {'accuracy': 0.9479301745635911, 'f1': 0.7119205298013245, 'precision': 0.8376623376623377, 'recall': 0.6190019193857965}}


In [25]:
import pickle

file_path = "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/SSL_Experiments/SimCLR_experiments/siclr_result/5D%_no_ssl.pkl"
with open(file_path, "wb") as file:
    pickle.dump(result_dict, file)

In [26]:
# import pickle

# # Load the dictionaries from the files
# with open("dictionary1_ssl.pkl", "rb") as file1, open("merged_dictionary_main.pkl", "rb") as file2:
#     dict1 = pickle.load(file1)
#     dict2 = pickle.load(file2)

# # Merge the dictionaries
# merged_dict = {**dict1, **dict2}

# # Save the merged dictionary to a file
# with open("merged_dictionary_main.pkl", "wb") as file:
#     pickle.dump(merged_dict, file)


In [27]:
import pickle

# Load the merged dictionary from the file
with open("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/SSL_Experiments/SimCLR_experiments/siclr_result/100+1_ssl.pkl", "rb") as file:
    merged_dict = pickle.load(file)

# Print the merged dictionary
print(merged_dict)
# 

{'100%+1% SimCLR': {'accuracy': 0.9135555555555556, 'f1': 0.5261875761266749, 'precision': 0.72, 'recall': 0.4145873320537428}}


In [28]:
import pandas as pd
resdf = pd.DataFrame(merged_dict)
resdf



,100%+1% SimCLR
accuracy,0.913556
f1,0.526188
precision,0.720000
recall,0.414587


In [29]:
# #list to tensor
# y_pred = torch.tensor(y_pred)
# y_test = torch.tensor(y_test)
# def results(y_labels,y_pred,plot_confusion = False):
#     y_labels = y_labels.to(device) 
#     y_pred = y_pred.to(device)
#     print(f"Accuracy = {(y_labels == y_pred).float().mean()}")
#     if plot_confusion:
#         cm = confusion_matrix(y_labels.cpu(), y_pred.cpu())
#         unique_labels = np.unique(y_labels.cpu())
#         cm_display = ConfusionMatrixDisplay(cm, display_labels=unique_labels).plot(values_format='d', cmap='Blues')
#         plt.show()

In [30]:
# results(y_test,y_pred,plot_confusion = True)

In [31]:
# import pandas as pd

# # Assuming df is your DataFrame
# df = pd.read_csv("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/SSL_Experiments/patchfill_experiments/SSL_Result.csv")

# # Extract the '100% SSL' column and convert it to a dictionary
# ssl_dict = df.set_index('Results')['100% SSL'].to_dict()

# # Display the created dictionary
# ssl_dict={'100% ssl':{'accuracy': 0.913217, 'f1': 0.696559, 'precision': 0.797193, 'recall': 0.654645}}


In [32]:
# file_path = "dictionary2.pkl"

# # Open the file in write mode
# with open(file_path, "wb") as file:
#     # Write the dictionary to the file
#     pickle.dump(ssl_dict, file)

In [33]:
# import pickle

# # Load the dictionaries from the files
# with open("merged_dictionary.pkl", "rb") as file1, open("dictionary2.pkl", "rb") as file2:
#     dict1 = pickle.load(file1)
#     dict2 = pickle.load(file2)

# # Merge the dictionaries
# merged_dict = {**dict1, **dict2}

# # Save the merged dictionary to a file
# with open("merged_dictionary.pkl", "wb") as file:
#     pickle.dump(merged_dict, file)

In [34]:
# resdf = pd.DataFrame(merged_dict)
# resdf
